In [1]:
import pyspark
from pyspark.sql import SparkSession

#Create SparkSession
spark = SparkSession.builder \
       .master("local[*]") \
       .appName("spark_juracy") \
       .config("spark.driver.memory", "80g")\
       .config("spark.executor.memory", "50g")\
       .config("spark.dirver.maxResultSize", "50g")\
       .getOrCreate()
sc = spark.sparkContext
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

/home/juracybertoldo2/.local/lib/python3.8/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
import glob
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:
sc.setCheckpointDir("/dados10t/datalake/raw/checkpoint_dir/")

### gerar tabela de drogas com dados de vacinação 
#### csv para mapeamento com nomes das vacinas originais

In [3]:
vac  = spark.read.parquet("/dados10t/datalake/standard/omop/data_with_ids/vacinacao07072022_pessoasunicas_5doses_v17-id_omop-v3.parquet/")

#vac  = spark.read.parquet("/dados10t/datalake/standard/vigivac/vacinacao/vacinacao17032023_pessoasunicas_7doses_v17.parquet/")

In [5]:
vac.describe("visit_occurrence_id").show()

+-------+--------------------+
|summary| visit_occurrence_id|
+-------+--------------------+
|  count|           446217353|
|   mean|3.0849539100000024E8|
| stddev|1.2881185458015317E8|
|    min|            85386715|
|    max|           531604067|
+-------+--------------------+



In [6]:
vac = vac.withColumn("visit_occurrence_id",F.col("visit_occurrence_id").cast(IntegerType()) )

In [7]:
vac.describe("visit_occurrence_id").show()

+-------+--------------------+
|summary| visit_occurrence_id|
+-------+--------------------+
|  count|           446217353|
|   mean|3.0849539100000024E8|
| stddev|1.2881185458015317E8|
|    min|            85386715|
|    max|           531604067|
+-------+--------------------+



In [8]:
vac.printSchema()

root
 |-- d_sexo: string (nullable = true)
 |-- d_codibge: string (nullable = true)
 |-- d_idade: double (nullable = true)
 |-- d_racacor: string (nullable = true)
 |-- d_paciente_endereco_uf: string (nullable = true)
 |-- d_total_regitros_dedup: double (nullable = true)
 |-- d1_nome_vacina: string (nullable = true)
 |-- d1_data_aplicacao: date (nullable = true)
 |-- d1_estab_mun_codigo: string (nullable = true)
 |-- d1_estab_valor: string (nullable = true)
 |-- d1_vacina_grupoAtendimento_nome: string (nullable = true)
 |-- d1_vacina_categoria_nome: string (nullable = true)
 |-- d1_vacina_dose: string (nullable = true)
 |-- d2_nome_vacina: string (nullable = true)
 |-- d2_data_aplicacao: date (nullable = true)
 |-- d2_estab_mun_codigo: string (nullable = true)
 |-- d2_estab_valor: string (nullable = true)
 |-- d2_vacina_grupoAtendimento_nome: string (nullable = true)
 |-- d2_vacina_categoria_nome: string (nullable = true)
 |-- d2_vacina_dose: string (nullable = true)
 |-- d3_nome_vacin

In [9]:
vac.groupBy("d1_nome_vacina").count().orderBy(F.desc("count")).show(60,False)

+---------------------------------------------------------+---------+
|d1_nome_vacina                                           |count    |
+---------------------------------------------------------+---------+
|Covid-19-AstraZeneca                                     |167749014|
|Covid-19-Coronavac-Sinovac/Butantan                      |135072640|
|Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pharma/Pfizer|134948772|
|Vacina covid-19 - Ad26.COV2.S - Janssen-Cilag            |8446927  |
+---------------------------------------------------------+---------+



In [10]:
vac.count() #446217353

446217353

In [8]:
#vac.filter(F.col("id_vigvac") == "661425859668").take(4)

In [11]:
vac.select("id_vigvac").distinct().count() #183759941

183759941

In [12]:
vac.select("visit_occurrence_id").distinct().count() #446217353

446217353

In [13]:
vac.groupBy("inconsistente").count().orderBy(F.desc("count")).show(30,False) #25938 

+-------------+---------+
|inconsistente|count    |
+-------------+---------+
|null         |446134022|
|SIM          |83331    |
+-------------+---------+



In [14]:
vac = vac.filter(F.col("inconsistente").isNull())

In [15]:
vac_join = vac.select("id_vigvac","n_vacina","data_aplicacao","visit_occurrence_id")

In [16]:
vac.count() #446134022

446134022

In [17]:
vac_v1 = vac.drop("n_vacina","data_aplicacao","visit_occurrence_id").dropDuplicates(['id_vigvac'])

In [18]:
vac_v1.count() #183734003

183734003

In [19]:
vac_v1.printSchema()

root
 |-- d_sexo: string (nullable = true)
 |-- d_codibge: string (nullable = true)
 |-- d_idade: double (nullable = true)
 |-- d_racacor: string (nullable = true)
 |-- d_paciente_endereco_uf: string (nullable = true)
 |-- d_total_regitros_dedup: double (nullable = true)
 |-- d1_nome_vacina: string (nullable = true)
 |-- d1_data_aplicacao: date (nullable = true)
 |-- d1_estab_mun_codigo: string (nullable = true)
 |-- d1_estab_valor: string (nullable = true)
 |-- d1_vacina_grupoAtendimento_nome: string (nullable = true)
 |-- d1_vacina_categoria_nome: string (nullable = true)
 |-- d1_vacina_dose: string (nullable = true)
 |-- d2_nome_vacina: string (nullable = true)
 |-- d2_data_aplicacao: date (nullable = true)
 |-- d2_estab_mun_codigo: string (nullable = true)
 |-- d2_estab_valor: string (nullable = true)
 |-- d2_vacina_grupoAtendimento_nome: string (nullable = true)
 |-- d2_vacina_categoria_nome: string (nullable = true)
 |-- d2_vacina_dose: string (nullable = true)
 |-- d3_nome_vacin

In [20]:
#vac_v2 = vac.select("d1_nome_vacina","d1_data_aplicacao","d1_tipo_vacina","d2_nome_vacina","d2_data_aplicacao","d2_tipo_vacina","d3_nome_vacina","d3_data_aplicacao","d3_tipo_vacina", 'd4_nome_vacina', 'd4_data_aplicacao',"d4_tipo_vacina", 'd5_nome_vacina', 'd5_data_aplicacao',"d5_tipo_vacina","id_vigvac")
vac_v2 = vac_v1.select("d1_nome_vacina","d1_data_aplicacao","d2_nome_vacina","d2_data_aplicacao","d3_nome_vacina","d3_data_aplicacao", 'd4_nome_vacina', 'd4_data_aplicacao', 'd5_nome_vacina', 'd5_data_aplicacao',"id_vigvac")

#### teste usando stack

In [21]:
#df_transformed = vacteste_v2.selectExpr('id_vigvac', 'stack(5, d1_nome_vacina, d1_data_aplicacao, d1_tipo_vacina, d2_nome_vacina, d2_data_aplicacao,d2_tipo_vacina, d3_nome_vacina, d3_data_aplicacao,d3_tipo_vacina, d4_nome_vacina, d4_data_aplicacao,d4_tipo_vacina, d5_nome_vacina, d5_data_aplicacao, d5_tipo_vacina) as (nome, drug_exposure_start_date,tipo)')
df_transformed = vac_v2.selectExpr('id_vigvac', 'stack(5, d1_nome_vacina, d1_data_aplicacao, d2_nome_vacina, d2_data_aplicacao, d3_nome_vacina, d3_data_aplicacao, d4_nome_vacina, d4_data_aplicacao, d5_nome_vacina, d5_data_aplicacao) as (nome, drug_exposure_start_date)')

In [22]:
df_transformed.count() #918670015

918670015

In [23]:
df_transformed.filter(F.col("nome").isNotNull()).count() #446134022

446134022

In [24]:
df_transformed = df_transformed.filter(F.col("nome").isNotNull())

In [25]:
df_transformed.filter(F.col("id_vigvac") == "1142461303481").show(10,False)

+-------------+---------------------------------------------------------+------------------------+
|id_vigvac    |nome                                                     |drug_exposure_start_date|
+-------------+---------------------------------------------------------+------------------------+
|1142461303481|Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pharma/Pfizer|2021-08-14              |
|1142461303481|Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pharma/Pfizer|2021-10-09              |
|1142461303481|Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pharma/Pfizer|2022-02-09              |
+-------------+---------------------------------------------------------+------------------------+



In [26]:
vac_v2.filter(F.col("id_vigvac") == "1142461303481").show(10,False)

+---------------------------------------------------------+-----------------+---------------------------------------------------------+-----------------+---------------------------------------------------------+-----------------+--------------+-----------------+--------------+-----------------+-------------+
|d1_nome_vacina                                           |d1_data_aplicacao|d2_nome_vacina                                           |d2_data_aplicacao|d3_nome_vacina                                           |d3_data_aplicacao|d4_nome_vacina|d4_data_aplicacao|d5_nome_vacina|d5_data_aplicacao|id_vigvac    |
+---------------------------------------------------------+-----------------+---------------------------------------------------------+-----------------+---------------------------------------------------------+-----------------+--------------+-----------------+--------------+-----------------+-------------+
|Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pharma/Pfizer|2021-08-14 

In [27]:
#df_transformed = df_transformed.withColumn("drug_exposure_id",F.monotonically_increasing_id())

#df_transformed = df_transformed.withColumnRenamed("id_vigvac","person_id")


df_transformed = df_transformed.withColumn("drug_exposure_end_datetime",F.col("drug_exposure_start_date"))


df_transformed = df_transformed.withColumn("drug_type_concept_id",F.lit("32818") )   # 32818 é EHR administration record


df_transformed = df_transformed.withColumnRenamed("nome","drug_source_value")

In [28]:
"""



conceitos id por vacina

coronavac concept id 702673 CVX

astrazeneca concept id 724905 CVX

pfizer  concept id  37003436  RxNorm

jassen concept id  739906 RxNorm

"""
def vacina_to_conceito_id(vac,tipo_pfizer):  # usar para bancos de vacinacaod e 2023 em diante
    if vac:
        if vac == "Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pharma/Pfizer":
            if tipo_pfizer == "pediatrica":
                return 702117
            elif tipo_pfizer == "pediatrica_menor_5":
                return 779947
            return 37003436
        elif vac == "Vacina covid-19 - Ad26.COV2.S - Janssen-Cilag":
            return 739906
        elif vac == "Covid-19-Coronavac-Sinovac/Butantan":
            return 36941610
        elif vac == "Covid-19-AstraZeneca":
            return 724905
        
def vacina_to_conceito_id_2022(vac):  #usar para dados de vacinacao de 2022 e anterior
    if vac:
        if vac == "Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pharma/Pfizer":
          #  if tipo_pfizer == "pediatrica":
          #      return 702117
          #  elif tipo_pfizer == "pediatrica_menor_5":
          #      return 779947
            return 37003436
        elif vac == "Vacina covid-19 - Ad26.COV2.S - Janssen-Cilag":
            return 739906
        elif vac == "Covid-19-Coronavac-Sinovac/Butantan":
            return 36941610
        elif vac == "Covid-19-AstraZeneca":
            return 724905

udf_vacina_to_conceito_id = F.udf(vacina_to_conceito_id,IntegerType())

udf_vacina_to_conceito_id_2022= F.udf(vacina_to_conceito_id_2022,IntegerType())

In [29]:
df_transformed.groupBy("drug_source_value").count().orderBy(F.desc("drug_source_value")).show(30,False)

+---------------------------------------------------------+---------+
|drug_source_value                                        |count    |
+---------------------------------------------------------+---------+
|Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pharma/Pfizer|181277235|
|Vacina covid-19 - Ad26.COV2.S - Janssen-Cilag            |25182248 |
|Covid-19-Coronavac-Sinovac/Butantan                      |103615299|
|Covid-19-AstraZeneca                                     |136059240|
+---------------------------------------------------------+---------+



In [30]:
df_transformed.printSchema()

root
 |-- id_vigvac: long (nullable = true)
 |-- drug_source_value: string (nullable = true)
 |-- drug_exposure_start_date: date (nullable = true)
 |-- drug_exposure_end_datetime: date (nullable = true)
 |-- drug_type_concept_id: string (nullable = false)



In [31]:
df_transformed = df_transformed.withColumn("drug_concept_id",udf_vacina_to_conceito_id_2022("drug_source_value") )

In [32]:
df_transformed.groupBy("drug_concept_id","drug_source_value").count().orderBy(F.desc("drug_concept_id")).show(30,False)

+---------------+---------------------------------------------------------+---------+
|drug_concept_id|drug_source_value                                        |count    |
+---------------+---------------------------------------------------------+---------+
|37003436       |Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pharma/Pfizer|181277235|
|36941610       |Covid-19-Coronavac-Sinovac/Butantan                      |103615299|
|739906         |Vacina covid-19 - Ad26.COV2.S - Janssen-Cilag            |25182248 |
|724905         |Covid-19-AstraZeneca                                     |136059240|
+---------------+---------------------------------------------------------+---------+



In [33]:
df_transformed.show(100,False)

+---------+---------------------------------------------------------+------------------------+--------------------------+--------------------+---------------+
|id_vigvac|drug_source_value                                        |drug_exposure_start_date|drug_exposure_end_datetime|drug_type_concept_id|drug_concept_id|
+---------+---------------------------------------------------------+------------------------+--------------------------+--------------------+---------------+
|26       |Covid-19-Coronavac-Sinovac/Butantan                      |2022-03-16              |2022-03-16                |32818               |36941610       |
|29       |Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pharma/Pfizer|2021-08-20              |2021-08-20                |32818               |37003436       |
|29       |Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pharma/Pfizer|2021-11-04              |2021-11-04                |32818               |37003436       |
|474      |Vacina covid-19 - BNT162b2 - BioNTe

In [26]:
#df_transformed_v2 = df_transformed.drop("tipo")

In [34]:
df_transformed.printSchema()

root
 |-- id_vigvac: long (nullable = true)
 |-- drug_source_value: string (nullable = true)
 |-- drug_exposure_start_date: date (nullable = true)
 |-- drug_exposure_end_datetime: date (nullable = true)
 |-- drug_type_concept_id: string (nullable = false)
 |-- drug_concept_id: integer (nullable = true)



In [35]:
pacientes_v6 = spark.read.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_person.parquet/")

In [36]:
pacientes_v6.printSchema()

root
 |-- raca: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- id_vigvac: long (nullable = true)
 |-- gender_source_value: string (nullable = true)
 |-- race_source_value: string (nullable = true)
 |-- gender_concept_id: string (nullable = true)
 |-- race_concept_id: string (nullable = true)
 |-- year_of_birth: long (nullable = true)
 |-- month_of_birth: long (nullable = true)
 |-- day_of_birth: long (nullable = true)
 |-- birth_datetime: timestamp (nullable = true)
 |-- ethnicity_concept_id: string (nullable = true)
 |-- person_id: long (nullable = true)



In [37]:
pacientes_v6.describe("person_id").show()

+-------+--------------------+
|summary|           person_id|
+-------+--------------------+
|  count|           188537327|
|   mean|         9.4268663E7|
| stddev|5.4426038392208815E7|
|    min|                   0|
|    max|           188537326|
+-------+--------------------+



In [39]:
pacientes_join = pacientes_v6.select("person_id","id_vigvac")

In [50]:
pacientes_join = pacientes_join.withColumnRenamed("id_vigvac","id_vigivac")

In [51]:
pacientes_join.printSchema()

root
 |-- person_id: long (nullable = true)
 |-- id_vigivac: long (nullable = true)



In [41]:
df_transformed_v2 = df_transformed.withColumnRenamed("id_vigvac","id_vigivac")

In [42]:
df_transformed_v2.printSchema()

root
 |-- id_vigivac: long (nullable = true)
 |-- drug_source_value: string (nullable = true)
 |-- drug_exposure_start_date: date (nullable = true)
 |-- drug_exposure_end_datetime: date (nullable = true)
 |-- drug_type_concept_id: string (nullable = false)
 |-- drug_concept_id: integer (nullable = true)



In [43]:
df_transformed_v2.count() #446134022

446134022

In [44]:
pacientes_join.count() #187634532

188537327

In [52]:
df_transformed_v3 = df_transformed_v2.join(pacientes_join,['id_vigivac'],'inner').checkpoint()

In [53]:
df_transformed_v3.count() #446124123

446134022

In [54]:
#vac_join = vac.select("id_vigvac","n_vacina","data_aplicacao","visit_occurrence_id")
vac_join.take(2)

[Row(id_vigvac=463856468018, n_vacina='d1_data_aplicacao', data_aplicacao=datetime.date(2021, 2, 22), visit_occurrence_id=85386823),
 Row(id_vigvac=463856468099, n_vacina='d2_data_aplicacao', data_aplicacao=datetime.date(2021, 8, 16), visit_occurrence_id=85387026)]

In [55]:
vac_join = vac_join.withColumnRenamed("data_aplicacao","drug_exposure_start_date")


In [57]:
vac_join.printSchema()

root
 |-- id_vigvac: long (nullable = true)
 |-- n_vacina: string (nullable = true)
 |-- drug_exposure_start_date: date (nullable = true)
 |-- visit_occurrence_id: integer (nullable = true)



In [60]:
vac_join = vac_join.withColumnRenamed("id_vigvac","id_vigivac")

In [61]:
df_transformed_v4 = df_transformed_v3.join(vac_join ,['id_vigivac','drug_exposure_start_date']  , 'inner')

In [62]:
df_transformed_v4.count()

446134022

In [63]:
df_transformed_v4 = df_transformed_v4.withColumn("person_id",F.col("person_id").cast(IntegerType()) )

In [64]:
df_transformed_v4.select("visit_occurrence_id").distinct().count()

446134022

In [65]:
#abs(446124123 - 446124123)

In [66]:
df_transformed_v4.printSchema()

root
 |-- id_vigivac: long (nullable = true)
 |-- drug_exposure_start_date: date (nullable = true)
 |-- drug_source_value: string (nullable = true)
 |-- drug_exposure_end_datetime: date (nullable = true)
 |-- drug_type_concept_id: string (nullable = false)
 |-- drug_concept_id: integer (nullable = true)
 |-- person_id: integer (nullable = true)
 |-- n_vacina: string (nullable = true)
 |-- visit_occurrence_id: integer (nullable = true)



In [67]:
df_transformed_v5 = df_transformed_v4.drop("id_vigivac","n_vacina")

In [68]:
df_transformed_v5.printSchema()

root
 |-- drug_exposure_start_date: date (nullable = true)
 |-- drug_source_value: string (nullable = true)
 |-- drug_exposure_end_datetime: date (nullable = true)
 |-- drug_type_concept_id: string (nullable = false)
 |-- drug_concept_id: integer (nullable = true)
 |-- person_id: integer (nullable = true)
 |-- visit_occurrence_id: integer (nullable = true)



In [69]:
import time
start = time.time()

In [70]:
df_transformed_v6 = df_transformed_v5.rdd.zipWithIndex().toDF().select(F.col("_1.*"),F.col("_2").alias('drug_exposure_id'))

In [71]:
end = time.time()
print(end - start)

350.66943430900574


In [72]:
#df_transformed_v5= df_transformed_v5.withColumn("new_column",F.lit("ABC"))
#w = Window().partitionBy('new_column').orderBy(F.lit('A'))
#df_transformed_v6 = df_transformed_v5.withColumn("drug_exposure_id",F.row_number().over(w)).drop("new_column")

In [73]:
df_transformed_v6.printSchema()

root
 |-- drug_exposure_start_date: date (nullable = true)
 |-- drug_source_value: string (nullable = true)
 |-- drug_exposure_end_datetime: date (nullable = true)
 |-- drug_type_concept_id: string (nullable = true)
 |-- drug_concept_id: long (nullable = true)
 |-- person_id: long (nullable = true)
 |-- visit_occurrence_id: long (nullable = true)
 |-- drug_exposure_id: long (nullable = true)



In [74]:
df_transformed_v6 = df_transformed_v6.withColumn("drug_exposure_id",F.col("drug_exposure_id").cast(IntegerType()) )

In [75]:
df_transformed_v6.write.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_drug_exposure_v6.parquet")

In [76]:
check = spark.read.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_drug_exposure_v6.parquet/")

In [77]:
check.count()

446134022

In [78]:
check.printSchema()

root
 |-- drug_exposure_start_date: date (nullable = true)
 |-- drug_source_value: string (nullable = true)
 |-- drug_exposure_end_datetime: date (nullable = true)
 |-- drug_type_concept_id: string (nullable = true)
 |-- drug_concept_id: long (nullable = true)
 |-- person_id: long (nullable = true)
 |-- visit_occurrence_id: long (nullable = true)
 |-- drug_exposure_id: integer (nullable = true)

